In [ ]:
## Runs my implementation of Word2Vec
## The input is a text file plus some parameters 
## The pre-processing is performed by functions in TexttoDict

import numpy as np
import theano
import theano.tensor as T
import nltk
import string
from collections import defaultdict
from TexttoDict import *

rng = np.random

In [2]:
# N is the size of hidden layer
# inarr, outarr are the output of run_container
# inarr is the average of the bag of words
# out arr is the middle word in the bag

def W2Vtrain(inarr, outarr, hiddensize):
# V is the length of cutoff dictionary
    V=len(inarr[0])
    
    
# (x,y) are shared variables for (input, output)
    x = T.dvector("x") # a vector
    y = T.dvector("y") # a vector


# (W1,W2) are the weight matrices for input-hidden and hidden-output
    W1 = theano.shared(rng.randn(V,hiddensize), name="W1")   # a V x N matrix
    W2 = theano.shared(rng.randn(hiddensize,V), name="W2")   # a N x V matrix

# This evaluates the hidden and output layers
# This is a linear operation, no logistic function unlike neural nets
    vhidden = T.dot(x, W1)   
    vout = T.dot(vhidden,W2)   

# Entropy loss function (this is a scalar)
# y is the target word. x in the average of the bag
    Entropy = T.dot(y,vout) - T.log((T.exp(vout)).sum())

# Compute the gradient of the cost 
# w.r.t weight vector w and bias term b
# These will be updated on each training run
    gradW1, gradW2 = T.grad(Entropy, [W1, W2])

# The train function updates the weights and computes the entropy
    train = theano.function(
          inputs=[x,y],
          outputs=[Entropy],
          updates=((W1, W1 - 0.1 * gradW1),(W2, W2 - 0.1 * gradW2)))

# After a sufficient number of training runs, the entropy should vanish
    for i in range(len(inarr)):
        ent=train(inarr[i], outarr[i])
    
    return [W1.get_value(), ent,jj]

In [3]:
linecount("delorme.txt")

1599988

In [4]:
W2Vinput=w2vdatafromtext("delorme.txt",contsize=7,trainlines=1000,freqcutoff=1)

KeyboardInterrupt: 

In [ ]:
myW2Vdata=W2Vtrain(W2Vinput[1],W2Vinput[2],hiddensize=100)

In [ ]:
myW2Vdata[2]

In [ ]:
len(W2Vinput[0])
len(W2Vinput[1])

In [ ]:
import os
cwd = os.getcwd()
print cwd

In [ ]:
def w2vdatafromtext(tfile,contsize,trainlines,freqcutoff):
    textdict=textfile_worddict(tfile,lines=trainlines)
    textdictcut=dict_cutoff(textdict,freqcutoff)
    w2vdata=run_container("textfilesmall.txt",textdictcut,contsize,lines=trainlines)
    return [textdictcut,w2vdata[0],w2vdata[1]]

In [ ]:
textdict=textfile_worddict("delorme.txt",lines=100000)

In [ ]:
textdictcut=dict_cutoff(textdict,1)
w2vdata=run_container("delorme.txt",textdictcut,contsize,lines=trainlines)

